# Brick optimisation - simple proj rep calc

Created 08/04/2025

Objectives:
* Caclulate proj rep, check it's working so we can put into another object.

# Package imports

In [2]:
import sys
sys.path.append('../')

In [3]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    multiply_stacked_unitaries_against_mps,
    inner_product_b_tensors
)

from SPTOptimization.Optimizers.MPSBrickSolver import(
    MPSBrickSolver
)

# Load data

In [6]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [7]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [8]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [9]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x162a2fd10>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x165c09e90>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x16791cb10>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x16797e150>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x1679df390>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x167a64850>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x167ac5610>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x167b06fd0>}

In [10]:
loaded_data[(True, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 4.10310779e-55, 3.20108804e-32, 4.41166527e-30,
       4.30652209e-31, 1.28028128e-32, 2.18979328e-32, 9.64766348e-32,
       6.39204896e-31, 4.19476267e-32, 7.51835939e-33, 1.56998498e-32,
       3.06132263e-29, 6.24832887e-32, 6.64529320e-32, 6.61192056e-33,
       3.02331734e-32, 1.97077344e-32, 3.97848871e-32, 1.65936555e-32,
       8.12962747e-32, 8.36180681e-32, 1.05455111e-32, 2.30695596e-32,
       4.15719467e-32, 3.32855567e-32, 1.02762007e-32, 1.46697913e-32,
       3.80115562e-31, 1.63756131e-32, 9.98608231e-33, 1.82981182e-32,
       1.73380216e-33, 6.00451123e-32, 1.33036658e-32, 2.79091135e-31,
       1.64291033e-32, 3.68006128e-33, 9.95622775e-32, 6.17421509e-32,
       2.70280235e-32, 3.01371615e-31, 2.35183074e-32, 1.17896355e-32,
       3.93686429e-31, 8.45721273e-32, 1.08863312e-32, 3.77353007e-31,
       9.67562785e-33, 1.99656454e-33, 6.40393616e-33, 2.78084102e-32,
       1.03150150e-31, 4.37579428e-33, 1.45445289e-32, 3.55938597e-32,
      

In [11]:
loaded_data[(False, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 3.26915052e-29, 3.33293732e-29, 0.00000000e+00,
       3.33293732e-29, 7.79030959e-29, 4.21824880e-29, 4.21824880e-29,
       5.96576060e-30, 1.14662165e-29, 3.59424750e-29, 8.18720523e-29,
       4.21824880e-29, 8.28796989e-29, 4.21824880e-29, 3.79670126e-29,
       4.21824880e-29, 8.18720523e-29, 1.37190923e-28, 0.00000000e+00,
       1.37190923e-28, 0.00000000e+00, 3.66111579e-29, 3.66111579e-29,
       3.86541844e-29, 3.33293732e-29, 0.00000000e+00, 0.00000000e+00,
       3.59424750e-29, 1.56000325e-28, 3.46235982e-29, 0.00000000e+00,
       3.59424750e-29, 3.59424750e-29, 3.72860037e-29, 0.00000000e+00,
       0.00000000e+00, 9.22258517e-29, 8.33234331e-30, 9.01057880e-29,
       3.46235982e-29, 4.43734259e-29, 3.14342582e-29, 3.14342582e-29,
       0.00000000e+00, 8.18720523e-29, 1.46709639e-29, 1.46709639e-29,
       1.46709639e-29, 8.98561875e-30, 0.00000000e+00, 8.01495006e-30,
       4.07526776e-29, 4.00470169e-29, 9.32150093e-30, 7.69262642e-29,
      

In [12]:
loaded_exps = dict()

for k, psi in loaded_data.items():
    exp = psi.expectation_value('N', sites=[101,])[0]
    loaded_exps[k] = exp

In [13]:
loaded_exps

{(False, (1, 1)): 0.4999999999999988,
 (True, (1, 1)): 0.4999999999999729,
 (True, (0, 1)): 0.49999999999999745,
 (False, (0, 1)): 0.4999999999999949,
 (False, (1, 0)): 0.49999999999998695,
 (True, (1, 0)): 0.49999999999999367,
 (True, (0, 0)): 6.403936160073908e-33,
 (False, (0, 0)): 0.0}

# Definitons

In [14]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [15]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [16]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [17]:
bosonic_npc_symmetries = [
    to_npc_array(X) for X in bosonic_np_symmetries
]

In [18]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [19]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [20]:
fermionic_np_symmetries = [np_I, np_JW]

In [21]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [22]:
npc_JW = fermionic_npc_symmetries[1]

In [23]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [24]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [25]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=50,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases[(*k, i, j)] = case

In [26]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

In [27]:
shifted_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(shifted_symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I]*1,
                right_boundary_unitaries=[np_I, np_00]*1
            )

            shifted_cases[(*k, i, j)] = case

In [28]:
for c in shifted_cases.values():
    c.compute_svd_approximate_expectation()

In [29]:
symmetric_cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                [s[1],] + s*49,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I, np_00],
                right_boundary_unitaries=[np_00, np_I, np_00]
            )

            symmetric_cases[(*k, i, j)] = case

In [30]:
for c in symmetric_cases.values():
    c.compute_svd_approximate_expectation()

In [31]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [32]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

# Sweep

In [31]:
block_width = 5
num_one_sided_iterations = 1
num_two_sided_iterations = 3
num_layers = 1
num_sites = 5

In [32]:
optimizer_dict = dict()

for k, v in cases.items():
    opt = MPSBrickSolver(
        v,
        num_sites,
        block_width,
        num_layers,
        num_one_sided_iterations,
        num_two_sided_iterations,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    )

    opt.optimise()

    optimizer_dict[k] = opt

In [33]:
final_exps = dict()

for k, v in optimizer_dict.items():
    final_exp = v.flatten_exps()[-1]
    final_exps[k] = final_exp

In [34]:
bad_cases = [
    k
    for k, v in final_exps.items()
    if v < 0.99
]

In [35]:
bad_cases

[]

# Test proj rep

Give a wavefunction, calculate proj rep.

## SPT functions

In [36]:
def calculate_proj_rep_phase(state_g, left_schmidt_values_g, unitary_h,
                             state_gh,
                             left_environment,
                             max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    prod_state, _ = multiply_stacked_unitaries_against_mps(
        unitary_h,
        state_g,
        left_schmidt_values_g,
        max_virtual_bond_dim
    )

    exp = inner_product_b_tensors(prod_state, state_gh, left_environment)

    return exp/(np.abs(exp))

In [37]:
def get_proj_rep_phases(mps_tensors, left_schmidt_values, unitaries,
                        left_environment):
    mps_dict = {
        k: v for k, v in zip(symmetry_labels[1:], mps_tensors)
    }

    schmidt_vals_dict = {
        k: v for k, v in zip(symmetry_labels[1:], left_schmidt_values)
    }

    unitaries_dict = {
        k: v for k, v in zip(symmetry_labels[1:], unitaries)
    }

    out_phases = list()

    for a, b, c in group_products:
        phase = calculate_proj_rep_phase(
            mps_dict[a],
            schmidt_vals_dict[a],
            unitaries_dict[b],
            mps_dict[c],
            left_environment,
            max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
        )

        out_phases.append(phase)

    return out_phases

## Group data definitions

In [38]:
from itertools import combinations

In [39]:
symmetry_labels = [
    'II',
    'IX',
    'XI',
    'XX'
]

In [40]:
symmetry_combination_labels = list(combinations(symmetry_labels, 2))

In [41]:
symmetry_combination_labels

[('II', 'IX'),
 ('II', 'XI'),
 ('II', 'XX'),
 ('IX', 'XI'),
 ('IX', 'XX'),
 ('XI', 'XX')]

In [42]:
e, a, b, c = symmetry_labels

In [43]:
group_products = [
    (a, b, c),
    (b, a, c),
    (a, c, b),
    (c, a, b),
    (c, b, a),
    (b, c, a)
]

## Test case 1

In [44]:
proj_rep = True
n1_pair = (1, 0)

# May need to treat more consistently later.
jw_string = False

In [45]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [46]:
test_opt = sub_opts[0]

In [47]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [48]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [49]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [50]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [51]:
left_environment = null_opt.right_side_left_symmetry_environment

In [52]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [53]:
proj_rep_phases

[(-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j), (-1+0j)]

These are wrong... Wait, need "gauge invariant" phases?

## Test case 2

In [54]:
proj_rep = True
n1_pair = (0, 0)

# May need to treat more consistently later.
jw_string = False

In [55]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [56]:
test_opt = sub_opts[0]

In [57]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [58]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [59]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [60]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [61]:
left_environment = null_opt.right_side_left_symmetry_environment

In [62]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [63]:
proj_rep_phases

[(-1+0j),
 (-0.9999999999999999+0j),
 (-1+0j),
 (-1+0j),
 (-1+0j),
 (-0.9999999999999999+0j)]

Better...! So why wrong when I have a non-trivial fermion homology?

## Test case 3

In [64]:
proj_rep = False
n1_pair = (1, 0)

# May need to treat more consistently later.
jw_string = False

In [65]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [66]:
test_opt = sub_opts[0]

In [67]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [68]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [69]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [70]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [71]:
left_environment = null_opt.right_side_left_symmetry_environment

In [72]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [73]:
proj_rep_phases

[(1+0j), (0.9999999999999999+0j), (1+0j), (1+0j), (1+0j), (1+0j)]

## Test case 4

In [74]:
proj_rep = False
n1_pair = (0, 0)

# May need to treat more consistently later.
jw_string = False

In [75]:
sub_opts = [
    optimizer_dict[(proj_rep, n1_pair, jw_string, i)]
    for i in range(4)
]

### Right side

In [76]:
test_opt = sub_opts[0]

In [77]:
mps_tensors = [
    opt.top_right_mps_tensors
    for opt in sub_opts
]

In [78]:
right_side_left_schmidt_values = [
    opt.current_right_side_left_schmidt_values
    for opt in sub_opts
]

In [79]:
unitaries = [
    opt.right_unitaries
    for opt in sub_opts
]

In [80]:
null_opt = optimizer_dict[(proj_rep, n1_pair, 0, 0)]

In [81]:
left_environment = null_opt.right_side_left_symmetry_environment

In [82]:
proj_rep_phases = get_proj_rep_phases(
    mps_tensors,
    right_side_left_schmidt_values,
    unitaries,
    left_environment
)

In [83]:
proj_rep_phases

[(1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j)]

# String order parameters

In [33]:
np_Z_01 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, -1]
])

np_Z_10 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, -1]
])

np_Z_11 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
])

In [34]:
bosonic_np_phase_symmetries = [
    np_00,
    np_Z_01,
    np_Z_10,
    np_Z_11
]

In [35]:
bosonic_npc_phase_symmetries = [
    to_npc_array(X) for X in bosonic_np_phase_symmetries
]

In [36]:
npc_Z_01 = to_npc_array(np_Z_01)
npc_Z_10 = to_npc_array(np_Z_10)
npc_Z_11 = to_npc_array(np_Z_11)

In [37]:
fermion_identity = fermionic_npc_symmetries[0]

In [38]:
symmetry_term = [npc_01, fermion_identity]*50

In [39]:
string_order_term_1 = [npc_00, fermion_identity] + symmetry_term + [npc_00,]
string_order_term_2 = [npc_Z_01, fermion_identity] + symmetry_term + [npc_Z_01,]

In [50]:
string_order_params = dict()

n = 0

for l, v in loaded_data.items():
    for i, s in enumerate(bosonic_npc_symmetries):
        for j, b1 in enumerate(bosonic_npc_phase_symmetries):
            for k, b2 in enumerate(bosonic_npc_phase_symmetries):

                term = [b1, fermion_identity] + [s, fermion_identity]*5 + [b2,]
                exp = v.expectation_value_multi_sites(term, 50)

                string_order_params[(*l, i, j, k)] = exp
                n+=1

In [52]:
large_string_order_params = {
    k: v
    for k, v in string_order_params.items()
    if v > 0.1
}

In [53]:
large_string_order_params

{(False, (1, 1), 0, 0, 0): array(1.),
 (False, (1, 1), 3, 0, 0): array(1.),
 (True, (1, 1), 0, 0, 0): array(1.),
 (True, (0, 1), 0, 0, 0): array(1.),
 (False, (0, 1), 0, 0, 0): array(1.),
 (False, (0, 1), 1, 0, 0): array(1.),
 (False, (1, 0), 0, 0, 0): array(1.),
 (False, (1, 0), 2, 0, 0): array(1.),
 (True, (1, 0), 0, 0, 0): array(1.),
 (True, (0, 0), 0, 0, 0): array(1.),
 (False, (0, 0), 0, 0, 0): array(1.),
 (False, (0, 0), 1, 0, 0): array(1.),
 (False, (0, 0), 2, 0, 0): array(1.),
 (False, (0, 0), 3, 0, 0): array(1.)}

In [54]:
{
    k: v
    for k, v in large_string_order_params.items()
    if not k[0]
}

{(False, (1, 1), 0, 0, 0): array(1.),
 (False, (1, 1), 3, 0, 0): array(1.),
 (False, (0, 1), 0, 0, 0): array(1.),
 (False, (0, 1), 1, 0, 0): array(1.),
 (False, (1, 0), 0, 0, 0): array(1.),
 (False, (1, 0), 2, 0, 0): array(1.),
 (False, (0, 0), 0, 0, 0): array(1.),
 (False, (0, 0), 1, 0, 0): array(1.),
 (False, (0, 0), 2, 0, 0): array(1.),
 (False, (0, 0), 3, 0, 0): array(1.)}

In [57]:
np.all((-np_01 @ np_Z_01 @ np_01) == np_Z_01)

True

In [58]:
np.all((-np_10 @ np_Z_10 @ np_10) == np_Z_10)

True

In [63]:
np.all((np_11 @ np_Z_11 @ np_11) == np_Z_11)

True

In [64]:
np.all((-np_11 @ np_Z_10 @ np_11) == np_Z_10)

True

In [65]:
np.all((-np_11 @ np_Z_01 @ np_11) == np_Z_01)

True

In [66]:
np.all((np_01 @ np_Z_10 @ np_01) == np_Z_10)

True

In [67]:
np.all((np_10 @ np_Z_01 @ np_10) == np_Z_01)

True

In [68]:
string_order_params[(False, (0, 0), 1, 2, 2)]

array(1.2745034e-29)

In [69]:
string_order_params[(True, (0, 0), 1, 2, 2)]

array(-2.02661859e-27)

All looks good...

# Verify SPT phase with environments

In [106]:
test_cases = [
    cases[True, (0, 0), 0, i]
    for i in range(4)
]

In [107]:
test_case = test_cases[0]

In [108]:
test_case

In [109]:
test_case.right_projected_symmetry_state

<npc.Array shape=(4, 4) labels=['vR', 'vR*']>

In [110]:
def multiply_right_environments(env1, env2):
    out = npc.tensordot(
        env1,
        env2,
        [['vR',], ['vR*']]
    )

    return out

In [111]:
from functools import reduce

In [112]:
help(reduce)

Help on built-in function reduce in module _functools:

reduce(...)
    reduce(function, iterable[, initial]) -> value
    
    Apply a function of two arguments cumulatively to the items of a sequence
    or iterable, from left to right, so as to reduce the iterable to a single
    value.  For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
    ((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
    of the iterable in the calculation, and serves as a default when the
    iterable is empty.



In [113]:
def normalise_scaled_unitary(matrix):
    m = matrix

    tr = npc.tensordot(
        m,
        m.conj(),
        [['vR', 'vR*'], ['vR*', 'vR']]
    )

    dim = m.to_ndarray().shape[0]

    scale = np.sqrt(tr/dim)

    out = m/scale

    return out

In [114]:
def right_proj_rep_invariant_phase_from_environments(case0, case1, case2):
    v0 = case0.right_projected_symmetry_state

    v1 = case1.right_projected_symmetry_state
    v1 = npc.tensordot(
        v0.conj(),
        v1,
        [['vR',], ['vR*']]
    )
    v1 = normalise_scaled_unitary(v1)

    #print(np.round(npc.tensordot(v1, v1.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    v2 = case2.right_projected_symmetry_state
    v2 = npc.tensordot(
        v0.conj(),
        v2,
        [['vR',], ['vR*']]
    )
    v2 = normalise_scaled_unitary(v2)

    #print(np.round(npc.tensordot(v2, v2.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    t = reduce(
        multiply_right_environments,
        [v1, v2, v1.conj(), v2.conj()]
    )

    exp = npc.trace(t)
    phase = exp/np.abs(exp)

    return phase

In [115]:
right_proj_rep_invariant_phase_from_environments(
    test_cases[0],
    test_cases[1],
    test_cases[2]
)

(1+0j)

In [116]:
from itertools import combinations

In [117]:
case_pairs = list(combinations(test_cases[1:], 2))

In [119]:
phases = [
    right_proj_rep_invariant_phase_from_environments(test_cases[0], c1, c2)
    for c1, c2 in case_pairs
]

In [120]:
phases

[(1+0j), (1+0j), (1+0j)]

In [121]:
test_cases = [
    cases[False, (0, 0), 0, i]
    for i in range(4)
]

In [122]:
case_pairs = list(combinations(test_cases[1:], 2))

In [123]:
phases = [
    right_proj_rep_invariant_phase_from_environments(test_cases[0], c1, c2)
    for c1, c2 in case_pairs
]

In [124]:
phases

[(1+0j), (1+0j), (1+0j)]

In [125]:
test_cases = [
    cases[True, (1, 0), 0, i]
    for i in range(4)
]

In [126]:
case_pairs = list(combinations(test_cases[1:], 2))

In [127]:
phases = [
    right_proj_rep_invariant_phase_from_environments(test_cases[0], c1, c2)
    for c1, c2 in case_pairs
]

In [128]:
phases

[(1+0j), (1+0j), (1+0j)]

In [139]:
def right_proj_rep_invariant_phase_from_environments_1(case1, case2):
    v1 = case1.right_projected_symmetry_state
    v1 = normalise_scaled_unitary(v1)

    #print(np.round(npc.tensordot(v1, v1.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    v2 = case2.right_projected_symmetry_state

    v2 = normalise_scaled_unitary(v2)

    #print(np.round(npc.tensordot(v2, v2.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    t = reduce(
        multiply_right_environments,
        [v1, v2, v1.conj(), v2.conj()]
    )

    exp = npc.trace(t)
    phase = exp/np.abs(exp)

    return phase

In [155]:
test_cases = [
    cases[True, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [156]:
test_cases = [
    cases[False, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [157]:
test_cases = [
    cases[True, (1, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [158]:
test_cases = [
    shifted_cases[True, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [159]:
test_cases = [
    shifted_cases[False, (0, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

In [161]:
test_cases = [
    shifted_cases[True, (1, 0), 0, i]
    for i in range(4)
]

case_pairs = list(combinations(test_cases[1:], 2))

[
    right_proj_rep_invariant_phase_from_environments_1(c1, c2)
    for c1, c2 in case_pairs
]

[(1+0j), (1+0j), (1+0j)]

# Check entanglement spectrum

In [135]:
for k, v in loaded_data.items():
    print(f"{k} - {v.get_SR(101)}")

(False, (1, 1)) - [0.70710678 0.70710678]
(True, (1, 1)) - [0.5 0.5 0.5 0.5]
(True, (0, 1)) - [0.5 0.5 0.5 0.5]
(False, (0, 1)) - [0.70710678 0.70710678]
(False, (1, 0)) - [0.70710678 0.70710678]
(True, (1, 0)) - [0.5 0.5 0.5 0.5]
(True, (0, 0)) - [0.5 0.5 0.5 0.5]
(False, (0, 0)) - [1.]


In [136]:
for k, v in loaded_data.items():
    print(f"{k} - {v.get_SR(100)}")

(False, (1, 1)) - [0.70710678 0.70710678]
(True, (1, 1)) - [0.5 0.5 0.5 0.5]
(True, (0, 1)) - [0.5 0.5 0.5 0.5]
(False, (0, 1)) - [0.70710678 0.70710678]
(False, (1, 0)) - [0.70710678 0.70710678]
(True, (1, 0)) - [0.5 0.5 0.5 0.5]
(True, (0, 0)) - [0.5 0.5 0.5 0.5]
(False, (0, 0)) - [1.]


So there is something going on... The 2-fold degeneracy due to the fermion spt is unexpected for me.

# To-do:
Calculate string order parameters.